In [23]:
import psycopg2
#from config import config
from configparser import ConfigParser
import pandas as pd

In [38]:
game_results_23_df = pd.read_csv('game_results_2023.csv')
game_results_23_df.rename(columns={'Unnamed: 0': 'id'}, inplace = True)
game_results_23_df['id'] = game_results_23_df['visitor'] + game_results_23_df['home'] + game_results_23_df['dates'].apply(str)
game_results_23_df.set_index('id', inplace = True)

In [37]:
game_results_23_df

,id,visitor,home,dates,visitor_score,home_score
0,PHIBOS20221018,PHI,BOS,20221018,117,126
1,LALGSW20221018,LAL,GSW,20221018,109,123
2,ORLDET20221019,ORL,DET,20221019,109,113
3,WASIND20221019,WAS,IND,20221019,114,107
4,HOUATL20221019,HOU,ATL,20221019,107,117
...,...,...,...,...,...,...
1315,MIADEN20230601,MIA,DEN,20230601,93,104
1316,MIADEN20230604,MIA,DEN,20230604,111,108
1317,DENMIA20230607,DEN,MIA,20230607,109,94
1318,DENMIA20230609,DEN,MIA,20230609,108,95


In [39]:
tmp_df = "./game_results_2023_to_db.csv"
game_results_23_df.to_csv(tmp_df, index_label='id', header=False)
f = open(tmp_df, 'r')

In [40]:
def config(filename = 'database.ini', section = 'postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    db = {}

    if parser.has_section(section):
        params = parser.items(section)

        for param in params:
            db[param[0]] = param[1]

    else:
        raise Exception('Section{0} is not found in the {1} file'.format(section, filename))
    
    return(db)

#config()

In [ ]:
config()

In [42]:
def connect():
    connection = None
    try:
        params = config()
        #print(params)
        print('connecting to postgresql database...')
        connection = psycopg2.connect(**params)
       
        # create cursor
        cursor = connection.cursor()
        
        # example
        #print('postgresql database version: ')
        #crsr.execute('SELECT version()')
        #db_version = crsr.fetchone()
        #print(db_version)
        
        cursor.execute(open('create_game_results_table.sql', 'r').read())

        try:
            cursor.copy_from(f, 'game_results', sep=",")
            connection.commit()
            print("copy_from_file() done")
            cursor.close()
            #os.remove(tmp_df)
            
        except (Exception, psycopg2.DatabaseError) as error:
            #os.remove(tmp_df)
            print("Error: %s" % error)
            connection.rollback()
            cursor.close()
            return 1
    
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if connection is not None:
            connection.close()
            print('database connection terminated')

if __name__ == '__main__':
    connect()


connecting to postgresql database...
copy_from_file() done
database connection terminated
